In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections

In [2]:
conf = SparkConf().setMaster("local").setAppName("Coordinates")
sc = SparkContext(conf = conf)

In [4]:
lines = sc.textFile("train_full_parsed_clean3.1.csv")
header = lines.first()
header1= sc.parallelize([header])
linesnoheader = lines.subtract(header1)

In [5]:
def loaddirection(line):
    
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    return coordinates

In [6]:
direction = lines.flatMap(loaddirection)
direction

PythonRDD[22] at RDD at PythonRDD.scala:48

In [7]:
wordCounts = direction.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey(False)
results = wordCountsSorted.collect()

In [8]:
df=pd.DataFrame(results,columns=['count','coordinates'])

In [9]:
df['longitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[0])
df['latitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[1])

In [10]:
df[['count','longitude','latitude']].to_csv("newcounter.csv",index=False)